In [32]:
using JuMP
#Import HiGHS solver
using HiGHS

#Create a JuMP model named picframe1 that will be solved using the HiGHS solver
wiskey=Model(HiGHS.Optimizer);
wistype = [:std,:cho,:pri];
blends = [:scot,:johnny];
wisAvilable=Dict(zip(wistype,[1200,2500,2000]));
wisCost=Dict(zip(wistype,[4,5,7]));
demand=Dict(zip(blends,[1000,600]));
wisPrice=Dict(zip(blends,[6.8,5.7]));

@variable(wiskey,x[wistype,blends]>=0);
@variable(wiskey,y[wistype,blends]>=0);
@constraint(wiskey,yPSInf,y[:pri,:scot]>=0.6*sum(y[i,:scot] for i in wistype));
@constraint(wiskey,ySCSup,y[:std,:scot]<=0.2*sum(y[i,:scot] for i in wistype));
@constraint(wiskey,yPJInf,y[:pri,:johnny]>=0.15*sum(y[i,:johnny] for i in wistype));
@constraint(wiskey,ySJSup,y[:std,:johnny]<=0.6*sum(y[i,:johnny] for i in wistype));
@variable(wiskey,money[blends]>=0);
@constraint(wiskey,PSInf,x[:pri,:scot]>=0.6*sum(x[i,:scot] for i in wistype));
@constraint(wiskey,SCSup,x[:std,:scot]<=0.2*sum(x[i,:scot] for i in wistype));
@constraint(wiskey,PJInf,x[:pri,:johnny]>=0.15*sum(x[i,:johnny] for i in wistype));
@constraint(wiskey,SJSup,x[:std,:johnny]<=0.6*sum(x[i,:johnny] for i in wistype));
@constraint(wiskey, wisConstraint[i in wistype], sum(x[i,j] for j in blends)+
sum(y[i,j]*1.25*money[j] for j in blends)<=wisAvilable[i]);
@constraint(wiskey,prodcutInf[j in blends],sum(x[i,j] for i in wistype)<=demand[j]);

@objective(wiskey,Max, sum(x[i,j] for i in wistype for j in blends)*wisPrice[j]
-sum(x[i,j] for i in wistype for j in blends)*wisCost[i]-sum(money[i] for i in blends)+
money[:scot]*1.25*wisPrice[:scot]+money[:johnny]*wisPrice[:johnny]*1.25-sum(money[i] for i in blends));



print(wiskey)


ErrorException: Constraints of type MathOptInterface.ScalarQuadraticFunction{Float64}-in-MathOptInterface.LessThan{Float64} are not supported by the solver.

If you expected the solver to support your problem, you may have an error in your formulation. Otherwise, consider using a different solver.

The list of available solvers, along with the problem types they support, is available at https://jump.dev/JuMP.jl/stable/installation/#Supported-solvers.

In [23]:
optimize!(wiskey)
@show objective_value(wiskey)

Solving LP without presolve or with basis
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -1.1624986839e+01 Ph1: 0(0); Du: 2(11.625) 0s
          0    -1.1625000000e+01 Ph1: 0(0); Du: 2(11.625) 0s
          0     1.4000000000e+03 Pr: 0(0); Du: 2(11.625) 0s
Using EKK primal simplex solver
  Iteration        Objective     Infeasibilities num(sum)
          0     1.4000005374e+03 Pr: 0(0); Du: 2(11.625) 0s
          0     1.4000000000e+03 Pr: 0(0); Du: 2(11.625) 0s
Model   status      : Unbounded
Objective value     :  1.4000000000e+03
HiGHS run time      :          0.01
objective_value(wiskey) = 6.5


6.5